# 평가(기본 모델 - gemma-2b)

평가를 위해서 요약 결과를 확인하는 과정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip3 install -q -U transformers==4.38.2
!pip3 install -q -U datasets==2.18.0
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.9.0
!pip3 install -q -U trl==0.7.11
!pip3 install -q -U accelerate==0.27.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 110.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import random
import os

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments, pipeline, logging
from transformers import TrainingArguments, set_seed
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from datasets import load_dataset
from trl import SFTTrainer


import warnings
warnings.filterwarnings("ignore", category=UserWarning)

set_seed(42)

In [ ]:
# 불러올 모델 경로 - base model
new_model = "jental/gemma_2bnotchat"

In [ ]:
# 허깅페이스 토큰
HUGGINGFACE_AUTH_TOKEN = ''

In [ ]:
# bnb 환경 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
    ) # 4bit quantization

In [ ]:
# 토크나이저 불러오기
# tokenizer를 먼저 불러와야 한다
tokenizer = AutoTokenizer.from_pretrained(new_model,token=HUGGINGFACE_AUTH_TOKEN, padding_side="right")

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
# 모델 불러오기
model = AutoModelForCausalLM.from_pretrained(
        new_model,
        token = HUGGINGFACE_AUTH_TOKEN,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )
model.config.use_cache = True

adapter_config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/39.3M [00:00<?, ?B/s]

In [ ]:
model.eval()

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2048, out_features=256, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05,

In [ ]:
df_valid = pd.read_csv('/content/drive/MyDrive/new_project/project2/traindata/문서요약 텍스트/kobart_valid.tsv', sep='\t')

df_valid.head()

,news,summary
0,"[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, 체...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...
1,] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다. 바른미래당이 8일 연 ...,8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...
2,[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀. 지난 3일 한국이 세계 최초로 ...,지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...
3,] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소리...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...
4,"[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를 ...",주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...


In [ ]:
df_valid.shape

(30122, 2)

In [ ]:
prompt = ''
prompt = prompt + '<start_of_turn>user\n What are the summary in this news?: ' + df_valid['news'][5] + '<end_of_turn>\n'
prompt = prompt + '<start_of_turn>model\n the summary of the news in korean:'  '<end_of_turn>\n'

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

JTBC 손석희 대표이사의 '동승자' 뉴스가 공개됨에 따라, '당신이 믿었던 페이크'는 JTBC 손석희 대표이사의 '동승자' 뉴스를 추적하고 있으며, '당신이 믿었던 페이크'는 인터넷을 통해 진실을 추적하는 '서쳐(Searcher)'가 범람하는 가짜 뉴스의 실체를 파헤치는 프로그램이다.








 patrie, 문학, 영화 등 각종 문화 콘텐츠를 활용하여 사회 현안 문제를 조명하는 TV프로그램 '당신이 믿었던 페이크'(연출 박성현)가 오늘(9월 8일) 첫 방송된다. '당신이 믿었던 페이크'는 인터넷을 통해 진실을 추적하는 '서쳐(Searcher)'가 범람하는 가짜 뉴스의 실체를 파헤치는 프로그램이다. '당신이 믿었던 페이크'는 인터넷을 통해 진실을 추적하는 '


In [ ]:
news_num = 400
prompt = ''
prompt = prompt + '<start_of_turn>user\n What are the summary in this news?: ' + df_valid.iloc[news_num].news + '<end_of_turn>\n'
prompt = prompt + '<start_of_turn>model\n the summary of the news in korean:' + '<end_of_turn>\n'

In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**input_ids, max_new_tokens=512)
print(tokenizer.decode(outputs[0]).split('model')[1].replace('<eos>', '').strip())

the summary of the news in korean:<end_of_turn>
인간형 로봇(휴보)를 개발한 벤처기업 레인보우로보틱스가 8월 코스닥시장에 입성할 것으로 보인다. 이 회사는 지난해 매출 11억원에 영업손실 33억원, 순손실 32억원을 냈다. 이 회사는 지난해 기술특례 상장을 시도했지만 상장예비심사를 자진 철회하고 이번에 재도전하게 됐다.<end_of_turn>
<start_of_turn>


In [ ]:
cnt = 0
step1 = 1000
for i in range(0, len(df_valid), step1):
    cnt += 1
    if i != 30000:
        globals()[f'part{cnt}'] = df_valid[i:i+step1]
    else :
        globals()[f'part{cnt}'] = df_valid[i:]

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)


for data_num in range(3, 4):
    results = []
    data = globals()[f'part{data_num}']

    for text_num, text in enumerate(tqdm(data['news'])):

        prompt = ''
        prompt = prompt + '<start_of_turn>user\n What are the summary in this news?: ' + text + '<end_of_turn>\n'
        prompt = prompt + '<start_of_turn>model\n the summary of the news in korean:' + '<end_of_turn>\n'

        input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(**input_ids, max_new_tokens=512)


        results.append({'text' : text,
                        # 'origin_summary' : data['summary'][text_num],
                        'summary' : tokenizer.decode(outputs[0]).split('model')[1]
                        })
    globals()[f'df_result1_part{data_num}'] = pd.DataFrame(results)

    globals()[f'df_result1_part{data_num}'].to_csv(f'/content/drive/MyDrive/new_project/project2/test_result/model_2bnochat/result1_{data_num}.csv', index = False)
    globals()[f'df_result1_part{data_num}'].to_excel(f'/content/drive/MyDrive/new_project/project2/test_result/model_2bnochat/result1_{data_num}.xlsx', index = False)

100%|██████████| 1000/1000 [10:35:44<00:00, 38.14s/it]


In [ ]:
results

[{'text': '산청딸기는 산청군의 연간 820억원의 대표적인 높은 수익작물로 산청의 지리산이라는 좋은조건을 가진 청적지역이기에 고품질의 딸기를 생산해내고 있으며, 군은 산청딸기에 대한 명품화 사업과 홍보행사를 추진하면서 제품을 알리고 있다.',
  'summary': '\n the summary of the news in korean:<end_of_turn>\n산청딸기는 산청군의 연간 820억원의 대표적인 높은 수익작물로 산청의 지리산이라는 좋은조건을 가진 청적지역이기에 고품질의 딸기를 생산해내고 있으며, 군은 산청딸기에 대한 명품화 사업과 홍보행사를 추진하면서 제품을 알리고 있다.<end_of_turn>\n<start_of_turn>'},
 {'text': '충북 단양군이 귀농귀촌 도시 1번지로 손꼽히고 있으며 이는 단양군 전체 인구의 약 10%를 차지하는 비율로 농촌에 활력이 되고 있으며 단양군은 농기계 구입지원, 농가주택수리비 지원등 맞춤형 지원정책으로 귀농인의 정착을 돕고있다.',
  'summary': '\n the summary of the news in korean:<end_of_turn>\n충북 단양군은 귀농귀촌 도시 1번지로 손꼽히고 있으며 이는 단양군 전체 인구의 약 10%를 차지하는 비율로 농촌에 활력이 되고 있으며 단양군은 농기계 구입지원, 농가주택수리비 지원등 맞춤형 지원정책으로 귀농인의 정착을 돕고있다.<end_of_turn>\n<start_of_turn>'},
 {'text': '포장이사 서비스 yes2404가 이사전문가인 소사장이라는 명함을 가진 사장들이 직접 팀을 구성해 이사서비스를 제공하는 직거래 시스템을 통해 수수료나 소개비를 없애 이사실명제와 표준요금제, 품질평가시스템 제도를 도입, 고객들로부터 좋은 반응을 얻고 있는데 박해돈회장은 고품질을 목표로 다양한 서비스를 도입, 국내 이사문화를 한 차원 업그레이드 시켰다고 말했다.',
  'summary': '\n the summary of the news in korean:<

In [ ]:
text_num

In [ ]:
results